## Importing Libraries and Datasets:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [2]:
match = pd.read_csv('F:/Data Science/Data Sets/IPL Dataset/matches.csv')
delivery = pd.read_csv('F:/Data Science/Data Sets/IPL Dataset/deliveries.csv')

In [3]:
match.head(2)

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN


In [4]:
match.shape

(756, 18)

- So basically 'match' DataFrame contains complete information about each match palyed in all seasons from 2008 to 2019.
- Here for our model important columns will be 'id', 'city', 'team1', 'team2', 'dl_applied' and '	winner'. 

In [5]:
delivery.head(5)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [6]:
delivery.shape

(179078, 21)

- Here 'delivery' DataFrame contain information about each ball played in a match, for all 756 matchesh palyed in IPL from 2008 to 2019.
- So basically, here we are getting ball by ball information for each match in IPL.

## Preprocessing

% At final dataset, to predict output we need 10 columns
- 1) batting_team: team batting for the second inning of the match
- 2) bowling_team: team bowling for the second inning of the match
- 3) city: The city at which game is being played.
- 4) run_left: We require a column which can tell us how many runs are required to win the match, after each ball.
- 5) balls_left: We require a column which can tell us how many balls are left to win the match, after each ball.
- 6) Wickets_left: We require a column which can tell us how many Wickets are left to win the match, after each ball.
- 7) total_runs_x: The target score(runs) which is to be done to win the match.
- 8) CRR: current run rate of the match
- 9) RRR: Required run rate of the match to win
- 10) result: The actual result of the match.

### Step 1:

- Here we need to calculate total score(runs) of both team for all two innings.
- We will do this by applying groupby method.

In [7]:
total_score_df = delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()
total_score_df.head(6)

,match_id,inning,total_runs
0,1,1,207
1,1,2,172
2,2,1,184
3,2,2,187
4,3,1,183
5,3,2,184


In [8]:
total_score_df.shape

(1528, 3)

- Now we just need to keep values of score for first innings only
- This is because by using this first inning data and by tanking cuurent score of second inning data as input from user we need to predict probability score for each team.

In [9]:
total_score_df = total_score_df[total_score_df['inning'] == 1]
total_score_df.head()

,match_id,inning,total_runs
0,1,1,207
2,2,1,184
4,3,1,183
6,4,1,163
8,5,1,157


In [10]:
total_score_df.shape

(756, 3)

- Now we need to merge this 'total_score_df' data with 'match' dataset to get complete details of each match in the IPL

In [11]:
match_df = match.merge(total_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')
match_df.head(2)

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,match_id,total_runs
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN,1,207
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN,2,184


- Now some teams in this dataset are not playing currently in IPL, so we need to remove those team entries(rows) from the dataset.
- We need to remove ['Gujarat Lions', 'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants', 'Rising Pune Supergiant'] teams from the dataset.
- 'Delhi Daredevils' is renamed as 'Delhi Capitals' and 'Deccan Chargers' is renamed as 'Sunrisers Hyderabad', so we need to rename them in dataset also.

In [12]:
match_df['team1'].unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

In [13]:
teams = [
    'Sunrisers Hyderabad', 
    'Mumbai Indians', 
    'Royal Challengers Bangalore', 
    'Kolkata Knight Riders', 
    'Kings XI Punjab', 
    'Chennai Super Kings', 
    'Rajasthan Royals', 
    'Delhi Capitals'
]

In [14]:
match_df['team1'] = match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2'] = match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['team1'] = match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [15]:
match_df = match_df[match_df['team1'].isin(teams)]
match_df = match_df[match_df['team2'].isin(teams)]

In [16]:
match_df.shape

(641, 20)

- There are some matches which are affected by rain(bad weather) and whose end result is decided by duckworth luis method and this is mentioned in the above dataset by 'dl_applied' column.
- We only need to keep the data from matches where the result is not decided by duckworth luis method.

In [17]:
match_df['dl_applied'].value_counts()

0    626
1     15
Name: dl_applied, dtype: int64

In [18]:
match_df.shape

(641, 20)

In [19]:
match_df = match_df[match_df['dl_applied'] == 0]

In [20]:
match_df.shape

(626, 20)

In [21]:
match_df.columns

Index(['id', 'Season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3', 'match_id', 'total_runs'],
      dtype='object')

- Now we need to remove columns from 'match_df' dataframe which are not required for further analysis and prediction.
- Here we only need ['city', 'winner', 'match_id', 'total_runs'] from 'match_df' dataframe

In [22]:
match_df = match_df[['match_id', 'city', 'winner', 'total_runs']]
match_df.head(2)

,match_id,city,winner,total_runs
0,1,Hyderabad,Sunrisers Hyderabad,207
4,5,Bangalore,Royal Challengers Bangalore,157


- Now we will merge this 'match_df' dataframe with 'delivery' dataframe.

In [23]:
delivery_df = match_df.merge(delivery,on='match_id')
delivery_df.head(5)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder
0,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [24]:
delivery_df.shape

(149578, 24)

- Here, We got the dataset for each ball how game is played in both innings.
- Now we need the dataset where 'inning' column is 2, because we need this data to calculate output percentage prediction.

In [25]:
delivery_df = delivery_df[delivery_df['inning'] == 2]

In [26]:
delivery_df.shape

(72413, 24)

% At final dataset, to predict output we need 10 columns
- 1) batting_team: team batting for the second inning of the match
- 2) bowling_team: team bowling for the second inning of the match
- 3) city: The city at which game is being played.
- 4) run_left: We require a column which can tell us how many runs are required to win the match, after each ball.
- 5) balls_left: We require a column which can tell us how many balls are left to win the match, after each ball.
- 6) Wickets_left: We require a column which can tell us how many Wickets are left to win the match, after each ball.
- 7) total_runs_x: The target score(runs) which is to be done to win the match.
- 8) CRR: current run rate of the match
- 9) RRR: Required run rate of the match to win
- 10) result: The actual result of the match.

- From these few columns like 'batting_team', 'bowling team' and 'city' we can take directly from "delivery_df" DataFrame.
- Reamining all columns we need to calculate by ourself.

#### Checkpoint 1:

In [27]:
df = delivery_df.copy()

### 1) Creating 'runs_left' column:

In [28]:
df.head(2)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,0,0,0,1,0,1,NaN,NaN,NaN
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [29]:
df.groupby(['match_id']).cumsum()['total_runs_y']

125         1
126         1
127         1
128         3
129         7
         ... 
149573    152
149574    154
149575    155
149576    157
149577    157
Name: total_runs_y, Length: 72413, dtype: int64

In [30]:
df['current_score'] = df.groupby(['match_id']).cumsum()['total_runs_y']
df.head(2)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,0,0,1,0,1,NaN,NaN,NaN,1
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,0,0,NaN,NaN,NaN,1


In [31]:
df ['target'] = df['total_runs_x'] + 1

In [32]:
df['runs_left'] = df['target'] - df['current_score']
df.head(2)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,target,runs_left
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,1,0,1,NaN,NaN,NaN,1,208,207
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,NaN,NaN,NaN,1,208,207


### 2) Creating 'balls_left' column:

In [33]:
df['ball'].head(10)

125    1
126    2
127    3
128    4
129    5
130    6
131    1
132    2
133    3
134    4
Name: ball, dtype: int64

In [34]:
126 - (df['over']*6 + df['ball'])

125       119
126       118
127       117
128       116
129       115
         ... 
149573      4
149574      3
149575      2
149576      1
149577      0
Length: 72413, dtype: int64

In [35]:
df['balls_left'] = 126 - (df['over']*6 + df['ball'])

### 3) Creating 'wickets_left' column:

In [36]:
df['player_dismissed'] = df['player_dismissed'].fillna("0")

In [37]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda X: X if X == "0" else "1")

In [38]:
df['player_dismissed'] = df['player_dismissed'].astype('int')

In [39]:
wickets = df.groupby('match_id').cumsum()['player_dismissed'].values
wickets

array([0, 0, 0, ..., 6, 6, 7])

In [40]:
df['wickets_left'] = 10 - wickets
df.head(2)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,target,runs_left,balls_left,wickets_left
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,1,0,NaN,NaN,1,208,207,119,10
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,NaN,NaN,1,208,207,118,10


### 4) Creating 'CRR' (current run rate) column:
crr = runs/overs

In [41]:
df['ball_num_as_over'] = ((120 - df['balls_left'])/6) 

In [42]:
df['crr'] = df['current_score']/df['ball_num_as_over']
df['crr']

125       6.000000
126       3.000000
127       2.000000
128       4.500000
129       8.400000
            ...   
149573    7.862069
149574    7.897436
149575    7.881356
149576    7.915966
149577    7.850000
Name: crr, Length: 72413, dtype: float64

### 5) Creating RRR (required run rate) column:

In [43]:
df['rrr'] = (df['runs_left']*6/df['balls_left'])
df['rrr']

125       10.436975
126       10.525424
127       10.615385
128       10.603448
129       10.486957
            ...    
149573     1.500000
149574    -2.000000
149575    -6.000000
149576   -24.000000
149577         -inf
Name: rrr, Length: 72413, dtype: float64

### 6) Creating 'result' column:

In [44]:
df.head(2)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,dismissal_kind,fielder,current_score,target,runs_left,balls_left,wickets_left,ball_num_as_over,crr,rrr
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,NaN,NaN,1,208,207,119,10,0.166667,6.0,10.436975
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,NaN,NaN,1,208,207,118,10,0.333333,3.0,10.525424


In [45]:
df['batting_team'].unique()

array(['Royal Challengers Bangalore', 'Delhi Daredevils',
       'Mumbai Indians', 'Kings XI Punjab', 'Kolkata Knight Riders',
       'Sunrisers Hyderabad', 'Rajasthan Royals', 'Chennai Super Kings',
       'Deccan Chargers', 'Delhi Capitals'], dtype=object)

- 'Delhi Daredevils' is renamed as 'Delhi Capitals' and 'Deccan Chargers' is renamed as 'Sunrisers Hyderabad', so we need to rename them in dataset also.
- Here in 'batting_team' and 'bowling_team' columns we have to rename.

In [46]:
df['batting_team'] = df['batting_team'].str.replace('Delhi Daredevils','Delhi Capitals')
df['bowling_team'] = df['bowling_team'].str.replace('Delhi Daredevils','Delhi Capitals')

df['batting_team'] = df['batting_team'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
df['bowling_team'] = df['bowling_team'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [47]:
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

In [48]:
df['result'] = df.apply(result,axis=1)
df['result']

125       0
126       0
127       0
128       0
129       0
         ..
149573    0
149574    0
149575    0
149576    0
149577    0
Name: result, Length: 72413, dtype: int64

## Getting Cleanned DataFrame:

In [49]:
df.columns

Index(['match_id', 'city', 'winner', 'total_runs_x', 'inning', 'batting_team',
       'bowling_team', 'over', 'ball', 'batsman', 'non_striker', 'bowler',
       'is_super_over', 'wide_runs', 'bye_runs', 'legbye_runs', 'noball_runs',
       'penalty_runs', 'batsman_runs', 'extra_runs', 'total_runs_y',
       'player_dismissed', 'dismissal_kind', 'fielder', 'current_score',
       'target', 'runs_left', 'balls_left', 'wickets_left', 'ball_num_as_over',
       'crr', 'rrr', 'result'],
      dtype='object')

In [50]:
df_final = df[['batting_team','bowling_team','city','runs_left', 'balls_left', 'wickets_left','target','crr', 'rrr', 'result']]

In [51]:
df_final.head(2)

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,target,crr,rrr,result
125,Royal Challengers Bangalore,Sunrisers Hyderabad,Hyderabad,207,119,10,208,6.0,10.436975,0
126,Royal Challengers Bangalore,Sunrisers Hyderabad,Hyderabad,207,118,10,208,3.0,10.525424,0


### Shuffling dataset for training:

In [52]:
df_final = df_final.sample(df_final.shape[0])
df_final.head()

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,target,crr,rrr,result
62958,Royal Challengers Bangalore,Kings XI Punjab,Chandigarh,145,95,8,164,4.560000,9.157895,1
22050,Kings XI Punjab,Royal Challengers Bangalore,Durban,108,73,9,169,7.787234,8.876712,1
32008,Kings XI Punjab,Chennai Super Kings,Durban,105,103,9,117,4.235294,6.116505,0
53331,Sunrisers Hyderabad,Chennai Super Kings,Chennai,23,8,5,166,7.660714,17.250000,0
65407,Mumbai Indians,Sunrisers Hyderabad,Mumbai,93,107,9,101,3.692308,5.214953,1


In [53]:
df_final.reset_index(inplace = True, drop = True)
df_final.head()

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,target,crr,rrr,result
0,Royal Challengers Bangalore,Kings XI Punjab,Chandigarh,145,95,8,164,4.560000,9.157895,1
1,Kings XI Punjab,Royal Challengers Bangalore,Durban,108,73,9,169,7.787234,8.876712,1
2,Kings XI Punjab,Chennai Super Kings,Durban,105,103,9,117,4.235294,6.116505,0
3,Sunrisers Hyderabad,Chennai Super Kings,Chennai,23,8,5,166,7.660714,17.250000,0
4,Mumbai Indians,Sunrisers Hyderabad,Mumbai,93,107,9,101,3.692308,5.214953,1


In [54]:
df_final.to_csv('F:/Data Science/Data Sets/IPL Dataset/ipl_dataset_cleaned.csv')

In [55]:
df.head(2)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,fielder,current_score,target,runs_left,balls_left,wickets_left,ball_num_as_over,crr,rrr,result
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,NaN,1,208,207,119,10,0.166667,6.0,10.436975,0
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,NaN,1,208,207,118,10,0.333333,3.0,10.525424,0


In [56]:
df.to_csv('F:/Data Science/Data Sets/IPL Dataset/delivery_df_cleaned.csv')